In [ ]:
import pandas as pd 
import numpy as np
from joint_pose_vocab import vocab_dict
import matplotlib.pyplot as plt
import os
import cv2
import mediapipe as mp
import time 
import traceback
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier as KNN 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.svm import SVC
import pickle


TRAIN_SVM = True
TRAIN_FOREST = True
TRAIN_NN = True
TRAIN_KNN = True

os.chdir('../Dataset')
path = os.getcwd()

columns = ['class','l_shoulder','r_shoulder','l_arm','r_arm','l_hip','r_hip','l_knee','r_knee']

Y82_test = pd.read_csv(os.path.join(path,'Y82_testing_new.csv'), header=None)
Y82_train = pd.read_csv(os.path.join(path,'Y82_training_new.csv'), header=None)

L_test = pd.read_csv(os.path.join(path,'L_testing_new.csv'), header=None)
L_train = pd.read_csv(os.path.join(path,'L_training_new.csv'), header=None)

W2_test = pd.read_csv(os.path.join(path,'W2_testing_new.csv'), header=None)
W2_train = pd.read_csv(os.path.join(path,'W2_training_new.csv'), header=None)

Neutral_test = pd.read_csv(os.path.join(path,'Neutral_testing.csv'), header=None)
Neutral_train = pd.read_csv(os.path.join(path,'Neutral_training.csv'), header=None) 

combined_test = pd.concat([L_test, Y82_test, W2_test, Neutral_test])
combined_train = pd.concat([L_train, Y82_train, W2_train, Neutral_train])

combined_test.columns = columns 
combined_train.columns = columns 

combined_test['class'], classes = pd.factorize(combined_test['class'])
combined_train['class'], _ = pd.factorize(combined_train['class'])

''' Filtering out all the extra examples for the cobra class to make a more balanced data set '''
extra_test_cobra_rows = combined_test[combined_test['class'] == 1].sample(130)
extra_train_cobra_rows = combined_train[combined_train['class'] == 1].sample(275)

combined_test = combined_test.drop(extra_test_cobra_rows.index)
combined_train = combined_train.drop(extra_train_cobra_rows.index)

classes = list(classes)

''' These will be the list of classes when giving feedback due to the fact Tree and Warrior have directional variations'''
feedback_classes = classes + ['WarriorII_L', 'WarriorII_R', 'Tree_L_D', 'Tree_R_D', 'Tree_L_U', 'Tree_R_U']
feedback_classes.remove('Neutral')
feedback_classes.remove('Tree')
feedback_classes.remove('WarriorII')

joint_idx_map = {
        0 : 'Left Shoulder',
        1 : 'Right Shoulder',
        2 : 'Left Arm',
        3 : 'Right Arm',
        4 : 'Left Hip',
        5 : 'Right Hip',
        6 : 'Left Knee',
        7 : 'Right Knee'
    }

print(len(joint_idx_map))
print(len(classes))
print(len(columns))
print(len(feedback_classes))
print(len(combined_test))
print(len(combined_train))


In [ ]:
all_combined_df = pd.concat([Y82_train, Y82_test, L_test, L_train, W2_train, W2_test])
all_combined = pd.concat([combined_test, combined_train])

def split_features_labels(df):
    return df.drop('class', axis=1), df['class']
    
for i, c in enumerate(classes):
    print(f"Train: {c} - {len(combined_train[combined_train['class'] == i])}")
    print(f"Test: {c} - {len(combined_test[combined_test['class'] == i])}")
    print()

# Random Forest Classifer
# 88-89% Maybe 90?
## IDK anymore.. even this is at like 95% lol

In [ ]:
# from sklearn.model_selection import train_test_split // Don't need anymore

if TRAIN_FOREST:
  RANDOM_ORDER_DATA = True 
  MAX_ESTIMATORS = 100
  MAX_DEPTH = 8 

  if RANDOM_ORDER_DATA:
    mutated_train = combined_train.sample(frac=1)
  else:
    mutated_train = combined_train

  X_train, y_train = split_features_labels(mutated_train)
  X_test, y_test = split_features_labels(combined_test)


  forest_classifier = RandomForestClassifier()
  param_grid = {'n_estimators' : np.arange(1, MAX_ESTIMATORS),
                'max_depth' : np.arange(1, MAX_DEPTH),
              }

  forest_classifier_gscv = RandomizedSearchCV(forest_classifier, param_distributions=param_grid, cv=5, n_jobs=-1)

  #fit model to data
  forest_classifier_gscv.fit(X_train, y_train)

  MAX_DEPTH = forest_classifier_gscv.best_params_['max_depth']
  N_ESTIMATORS = forest_classifier_gscv.best_params_['n_estimators']

  best_forest = RandomForestClassifier(max_depth=MAX_DEPTH, n_estimators=N_ESTIMATORS)
  best_forest.fit(X_train, y_train)

  pred = best_forest.predict(X_test)
  cm = confusion_matrix(y_test, pred)
  display_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
  display_confusion_matrix.plot()
  print(classification_report(y_test, pred))

  # Dumping pre-trained model 
  # with open('/Users/mohamed/ZenAI-3YP/AppSite/backend/models/random_forest_92.pkl', 'wb') as f:
  #   pickle.dump(best_forest, f)

# KNN Classifer 
# Eh Around 88%


In [ ]:

if TRAIN_KNN:
    ''' Don't need to do this anymore, cause the dataset is already split (Didn't realise this) '''
    # from sklearn.model_selection import train_test_split
    # 80/20 Split of data, Doesn't randomize, Randomsplit ensures the proportion of classes is the same. 
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    RANDOM_ORDER_DATA = True 

    '''Shuffle data for better resuliting'''
    if RANDOM_ORDER_DATA:
        mutated_train = combined_train.sample(frac=1)
    else:
        mutated_train = combined_train

    X_train, y_train = split_features_labels(mutated_train)
    X_test, y_test = split_features_labels(combined_test)

    max_neighbours = int(np.sqrt(len(X_train)))
    knn_algorithms = ['kd_tree', 'brute', 'ball_tree']


    knn = KNN()

    param_grid = {'n_neighbors' : np.arange(1, max_neighbours),
                'algorithm' : knn_algorithms}

    # Using grid search cross validation to find the best value of K 
    knn_gscv = GridSearchCV(knn, param_grid, cv=5, n_jobs=-1)

    knn_gscv.fit(X_train, y_train)

    ALGORITHM = knn_gscv.best_params_['algorithm']
    N_NEIGHBORS = knn_gscv.best_params_['n_neighbors']

    best_KNN = KNN(algorithm=ALGORITHM, n_neighbors=N_NEIGHBORS)
    best_KNN.fit(X_train, y_train)

    print(f"Fitted KNN Classifer with {ALGORITHM=} and {N_NEIGHBORS=}")


    pred = best_KNN.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    display_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    display_confusion_matrix.plot()
    print(classification_report(y_test, pred))


# NN Classifier
## About 87-89% Acc - okay
## WOW WITH JUST A BIT OF DATACLEANING, GETTING SOME MORE EXAMPLES AND FIXING A BUG THE ACCURACY SHOT UP - 94% 

In [ ]:
if TRAIN_NN:
    GRID_SEARCH_PARAMS = True

    '''Shuffle data for better resuliting'''
    RANDOM_ORDER_DATA = True 
    if RANDOM_ORDER_DATA:
        mutated_train = combined_train.sample(frac=1)
    else:
        mutated_train = combined_train

    X_train, y_train = split_features_labels(mutated_train)
    X_test, y_test = split_features_labels(combined_test)

    # Define the parameter distributions to sample from
    param_dist = {
        'hidden_layer_sizes' : [(i, j, k) for i in range(1, 15) for j in range(1, 15) for k in range(1, 15)],
        'solver': ['adam', 'lbfgs'],
        'activation': ['relu', 'logistic'],
        'alpha' : [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    }

    # Initialize MLPClassifier with default values
    mlp = MLPClassifier(max_iter=1000)

    if GRID_SEARCH_PARAMS:
        random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, verbose=3)

        # Train the classifier on your data
        random_search.fit(X_train, y_train)

        # Get the best hyperparameters from the search
        best_params = random_search.best_params_
        print("Best solver: ", best_params['solver'])
        print("Best activation: ", best_params['activation'])
        print("Layers: ", best_params['hidden_layer_sizes'] )
        print("Alpha: ", best_params['alpha'])

        # Use the best hyperparameters to initialize the MLPClassifier
        best_mlp = MLPClassifier(solver=best_params['solver'], activation=best_params['activation'], alpha=best_params['alpha'], hidden_layer_sizes=best_params['hidden_layer_sizes'], max_iter=1000)

        # Train the MLPClassifier on the training data
        best_mlp.fit(X_train, y_train)
    else:
        ''' If you want to skip searching for params. Train NN with values:
        solver: adam
        activation: relu
        layers: 11, 14
        alpha: 0.0001
        '''
        best_mlp = MLPClassifier(solver='adam', activation='relu', alpha=0.0001, hidden_layer_sizes=(11, 14), max_iter=1000)
        best_mlp.fit(X_train, y_train)
        
    pred = best_mlp.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    display_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    display_confusion_matrix.plot()
    print(classification_report(y_test, pred))

In [ ]:
print("Best solver: ", best_params['solver'])
print("Best activation: ", best_params['activation'])
print("Layers: ", best_params['hidden_layer_sizes'] )
print("Alpha: ", best_params['alpha'])

#  Dumping pre-trained model 
with open('/Users/mohamed/ZenAI-3YP/AppSite/backend/models/nn_93.pkl', 'wb') as f:
    pickle.dump(best_mlp, f)

In [ ]:
# # Use the best hyperparameters to initialize the MLPClassifier
# best_mlp = MLPClassifier(solver=best_params['solver'], activation=best_params['activation'], alpha=best_params['alpha'], hidden_layer_sizes=best_params['hidden_layer_sizes'], max_iter=1000)

# # Train the MLPClassifier on the training data
# best_mlp.fit(X_train, y_train)


# SVC Grid Search Classifer
## Consistent 90% Sometimes 91%
## After replacing Warrior3 for Warrior2 (Is a hard pose to do, and W3 had too much overlap with the other classes + fixing a 20degree error in the pre-processing the accuracy gained went up by 5% !!)

In [ ]:
if TRAIN_SVM:
    RANDOM_ORDER_DATA = True 
    RANDOM_CV = False

    '''Shuffle data for better resuliting'''
    if RANDOM_ORDER_DATA:
        mutated_train = combined_train.sample(frac=1)
    else:
        mutated_train = combined_train

    X_train, y_train = split_features_labels(mutated_train)
    X_test, y_test = split_features_labels(combined_test)

    svm = SVC(kernel='rbf') 


    '''Tried search for gamma manually but it appears using scale is just better''' 
    n_features = X_train.shape[1]
    gamma_start = 1 / (n_features * max(X_train.var()))
    gamma_step = 0.005
    gamma_end = gamma_start + (10 * gamma_step)

    ### Doing Grid Search Now ###

    gamma_range = np.arange(gamma_start, gamma_end, gamma_step)
    C_range = [0.1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    # set the parameter grid for grid search
    param_grid = {
            'C': C_range,
            'gamma' : ['scale', 'auto']
        }

    # perform grid search
    if RANDOM_CV:
        grid = RandomizedSearchCV(svm, cv=5, param_distributions=param_grid, verbose=1)
    else:
        grid = GridSearchCV(svm, cv=5, param_grid=param_grid, verbose=1)
    grid.fit(X_train, y_train)

    # best parameters and score
    print("Best parameters:", grid.best_params_)
    print("Best Score: ", grid.best_score_)

    best_svc = SVC(kernel='rbf', C=grid.best_params_['C'], gamma=grid.best_params_['gamma'], probability=True)

    best_svc.fit(X_train, y_train)

    ''' Plotting '''
    pred = best_svc.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    display_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    display_confusion_matrix.plot()
    print(classification_report(y_test, pred))

In [ ]:
#  Dumping pre-trained model 
with open('/Users/mohamed/ZenAI-3YP/AppSite/backend/models/svm_93.pkl', 'wb') as f:
    pickle.dump(best_svc, f)

# Video & Classifer Integration

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

MIN_DETECTION_CONFIDENCE = 0.5
MIN_TRACKING_CONFIDENCE = 0.5

# All landmark except for hand and face specific
RelevantLandmarks = list(mp_pose.PoseLandmark)[11:17] + list(mp_pose.PoseLandmark)[23:29]

l_hip_landmark_angle_idx = (11,23,25)
r_hip_landmark_angle_idx = (12,24,26)

l_shoulder_landmark_angle_idx = (13,11,23)
r_shoulder_landmark_angle_idx = (14,12,24)

l_arm_landmark_angle_idx = (15,13,11)
r_arm_landmark_angle_idx = (16,14,12)

l_knee_landmark_angle_idx = (23,25,27)
r_knee_landmark_angle_idx = (24,26,28)

#Match idx of RelevantLandmarks 
angle_idxs_required = [
    l_shoulder_landmark_angle_idx,
    r_shoulder_landmark_angle_idx,
    
    l_arm_landmark_angle_idx,
    r_arm_landmark_angle_idx,
    
    l_hip_landmark_angle_idx,
    r_hip_landmark_angle_idx,
    
    l_knee_landmark_angle_idx,
    r_knee_landmark_angle_idx
]

skip_landmark = {
    mp_pose.PoseLandmark.RIGHT_ANKLE,
    mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.LEFT_WRIST
}

landmarkStr = {
    mp_pose.PoseLandmark.NOSE : "NOSE",
    mp_pose.PoseLandmark.LEFT_EYE_INNER : "LEFT_EYE_INNER",
    mp_pose.PoseLandmark.LEFT_EYE : "LEFT_EYE",
    mp_pose.PoseLandmark.LEFT_EYE_OUTER : "LEFT_EYE_OUTER",
    mp_pose.PoseLandmark.RIGHT_EYE_INNER : "RIGHT_EYE_INNER",
    mp_pose.PoseLandmark.RIGHT_EYE : "RIGHT_EYE",
    mp_pose.PoseLandmark.RIGHT_EYE_OUTER : "RIGHT_EYE_OUTER",
    mp_pose.PoseLandmark.LEFT_EAR : "LEFT_EAR",
    mp_pose.PoseLandmark.RIGHT_EAR : "RIGHT_EAR",
    mp_pose.PoseLandmark.MOUTH_LEFT : "MOUTH_LEFT",
    mp_pose.PoseLandmark.MOUTH_RIGHT : "MOUTH_RIGHT",
    mp_pose.PoseLandmark.LEFT_SHOULDER : "LEFT_SHOULDER",
    mp_pose.PoseLandmark.RIGHT_SHOULDER : "RIGHT_SHOULDER",
    mp_pose.PoseLandmark.LEFT_ELBOW : "LEFT_ELBOW",
    mp_pose.PoseLandmark.RIGHT_ELBOW : "RIGHT_ELBOW",
    mp_pose.PoseLandmark.LEFT_WRIST : "LEFT_WRIST",
    mp_pose.PoseLandmark.RIGHT_WRIST : "RIGHT_WRIST",
    mp_pose.PoseLandmark.LEFT_PINKY : "LEFT_PINKY",
    mp_pose.PoseLandmark.RIGHT_PINKY : "RIGHT_PINKY",
    mp_pose.PoseLandmark.LEFT_INDEX : "LEFT_INDEX",
    mp_pose.PoseLandmark.RIGHT_INDEX : "RIGHT_INDEX",
    mp_pose.PoseLandmark.LEFT_THUMB : "LEFT_THUMB",
    mp_pose.PoseLandmark.RIGHT_THUMB : "RIGHT_THUMB",
    mp_pose.PoseLandmark.LEFT_HIP : "LEFT_HIP",
    mp_pose.PoseLandmark.RIGHT_HIP : "RIGHT_HIP",
    mp_pose.PoseLandmark.LEFT_KNEE : "LEFT_KNEE",
    mp_pose.PoseLandmark.RIGHT_KNEE : "RIGHT_KNEE",
    mp_pose.PoseLandmark.LEFT_ANKLE : "LEFT_ANKLE",
    mp_pose.PoseLandmark.RIGHT_ANKLE : "RIGHT_ANKLE",
    mp_pose.PoseLandmark.LEFT_HEEL : "LEFT_HEEL",
    mp_pose.PoseLandmark.RIGHT_HEEL : "RIGHT_HEEL",
    mp_pose.PoseLandmark.LEFT_FOOT_INDEX : "LEFT_FOOT_INDEX",
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX : "RIGHT_FOOT_INDEX"
}

def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)    
    c = np.array(c)   
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
    
    return angle 

In [27]:

def classify_pose(example, classifer):
    example = pd.DataFrame(np.array(example).reshape(1, -1), columns=columns[1:])

    ''' Testing to see if this model thing may have been the issues, it seems not'''
    class Model:    
        def __init__(self, columns, classes):
            self.columns = columns 
            self.classes = classes 
            # os.chdir(os.path.dirname(os.path.abspath(__file__)))

        def reshape_data(self, example):
            # Reshape the input data so that it can be fed into the classifier
            return pd.DataFrame(np.array(example).reshape(1, -1), columns=self.columns[1:])
        
        def predict(self, example, clf):
            # Predict the output for the input example using the given classifier
            formatted_example = self.reshape_data(example)
            probabilty_classes = clf.predict_proba(formatted_example)

            # Convert the predicted probabilities to a list of tuples containing the class and probability
            prob_predicted_classes = [(self.classes[i], p) for i, p in enumerate(probabilty_classes[0])]
            
            # Get the highest predicted class 
            predicted_class, prob = max(prob_predicted_classes, key=lambda x: x[1])
            
            return predicted_class, prob, sorted(prob_predicted_classes, key=lambda x: x[1], reverse=True)

    
    class ZenSVM(Model):
        def __init__(self, columns, classes):
            super().__init__(columns, classes)
            with open('/Users/mohamed/ZenAI-3YP/AppSite/backend/models/svm_93.pkl', 'rb') as f:
                self.svm = pickle.load(f)

        def predict(self, example):
            # Predict the output for the input example using the support vector machine classifier
            return super().predict(example, self.svm)

    my_test = ZenSVM(columns, classes)

    if classifer == 'KNN':
        probabilty_classes = best_KNN.predict_proba(example)
    elif classifer == 'Forest':
        probabilty_classes = best_forest.predict_proba(example)
    elif classifer == 'SVM':
        return my_test.predict(example)
        # probabilty_classes = my_test.predict(example)
        # probabilty_classes = best_svc.predict_proba(example)
    elif classifer == 'NN':
        probabilty_classes = best_mlp.predict_proba(example)
    else:
        raise Exception("Please enter valid classifer. Currently only ('KNN' | 'Forest' | 'NN' | 'SVM')")

    
    prob_predicted_classes = [] 
    for class_idx, prob in enumerate(probabilty_classes[0]):
        prob_predicted_classes.append((classes[class_idx], prob))
        
    prob_predicted_classes.sort(key = lambda x: x[1], reverse=True)
    
    # Get the highest predicted class 
    predicted_class = prob_predicted_classes[0]
    
    return (predicted_class[0], predicted_class[1], sorted(prob_predicted_classes, key = lambda x: x[1], reverse=True))






fuckme = [
[176.63333484322465, 176.69176023867143, 4.240852428777295, 18.21368933796927, 168.4953493195391, 159.97941867050056, 167.75040890288562, 177.10118153965269],
[176.19083979842546, 178.94923799074203, 5.658966295840503, 16.368748527390544, 168.25464194005448, 158.948936404512, 168.5129309373331, 173.78868011132434],
[175.5072769880376, 178.3015615662657, 0.3688514385639763, 14.863182353678626, 168.12259283751035, 154.64349830851745, 176.81611311894514, 179.21535327735089],
[93.74943388039098, 103.6239618534756, 62.928446190219034, 54.035818090937354, 169.21321655306807, 158.46193278264218, 165.2305718392309, 179.9746721382498],
[77.65329004124364, 81.95068406348042, 88.36293006509158, 77.40610710950979, 169.7604060260831, 165.84953532292818, 168.32986774869426, 167.73956277259234],
[49.992076793078766, 55.46394643544669, 119.92281650914754, 106.41326913279653, 169.0570924555436, 163.84263035746534, 177.91275582554863, 176.51010356983923],
[48.53215222631013, 48.80294625195137, 144.61548010885306, 136.96070372389607, 161.54527648495485, 160.75364132128234, 177.5581649537558, 174.70438349262335],
[62.59678793826742, 61.23394673046768, 127.63261250452817, 132.10980379839313, 160.73628158151107, 170.36762242865896, 173.98306575347516, 170.85600886574036],
[74.92928503620203, 79.63816601705784, 118.90665456598171, 118.22276844613066, 167.99894612057417, 167.68010417788085, 168.2518088710176, 169.37546074597367],
[67.16629596922886, 72.38391191160007, 125.40533575423055, 125.62281019251961, 168.59073037278125, 170.09366874807492, 167.57994332511095, 173.20710172167816],
[54.713589592479394, 51.84148668913433, 143.54809173451898, 143.59157113454796, 172.0943395533507, 171.8892781141109, 170.1898028088484, 169.42309316280978],
[39.28999223746173, 41.09537598281285, 147.60664504638055, 149.04667908160533, 154.87866787857158, 164.250539174016, 151.9206686380666, 159.13302383155096],
[34.279127276556956, 34.45349507322961, 146.19801450737506, 151.23683077966015, 149.84643305607884, 162.20980510629516, 161.14913337561254, 160.28742000753056],
[34.283571366729575, 33.71092525650793, 160.78131406340273, 155.85090349593577, 171.08926271023844, 165.63540141248467, 162.50998754274903, 158.70857100366007],
[35.75857737618462, 35.2246666697044, 153.87542993530934, 152.2138497780821, 167.4295110949839, 166.46843150243782, 161.66462583961066, 161.0019979006867],
[34.960169705524336, 34.10187841715799, 154.86788287665837, 152.84988610524402, 166.74323123007125, 166.70862405246623, 161.98120099314033, 158.02722051404703],
[34.298208737148975, 34.16941250869246, 157.12143304839398, 152.73470400286587, 170.27183912891817, 164.80706017198315, 166.1044869656972, 163.0384330766529],
[33.70431044072494, 32.558762478225354, 155.4061088535178, 153.71691387357103, 166.25979071172262, 166.44019967768693, 165.58452171221379, 162.2574789462359],
[37.37290010052256, 31.489694753268516, 138.77513188287168, 146.77846630511462, 149.45501319604, 168.38903263691492, 141.7479629040584, 127.63818459358833],
[78.39674078109336, 71.2651706096516, 68.58702863437318, 75.23389002836566, 171.6551283587618, 168.13402089522955, 58.53390778355782, 56.572206515703385],
[61.76311412980057, 54.13560381077846, 89.16813340603338, 93.78126388627697, 174.48671118824774, 171.6014401672517, 109.34522469492657, 108.2084071536105],
[172.53301347114495, 179.38074439606623, 97.98498140147329, 22.752578793413615, 76.7979575280612, 144.6991672456756, 169.8496297460958, 161.3827612372873],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[158.71240201792924, 155.895560619486, 4.075568865227875, 8.773791531974798, 169.83671120888013, 150.73739542709836, 146.44375297095868, 144.54476425154297],
[154.110812356332, 151.6332152595511, 6.481013834319837, 2.39905869848321, 168.20063227940392, 149.45717192891152, 153.25605829026208, 136.91317620397186],
[170.26597979777375, 173.09748941880514, 1.5061132773956205, 6.9025698682775065, 170.56925685125046, 148.01616422954135, 177.0048332713033, 174.58702160457406],
[165.8927239649992, 171.1676071982569, 5.2437582957084246, 4.237977292676621, 167.84754471392063, 151.5804389561528, 176.13224053055458, 178.76156628259415],
[173.6293401567604, 171.12790359013042, 1.8474218096682509, 4.371542897090497, 167.57306628190116, 155.03325635550343, 178.8483769760877, 179.34058776916063],
[141.47417969608762, 153.46851809001134, 17.763699752020294, 6.591769539686361, 169.73623607558838, 155.53653339196867, 170.35289994066247, 175.110709192049],
[116.17661965931923, 122.81462708134373, 40.556581791590546, 32.79423100422199, 164.61163808452363, 160.73619169763705, 162.95016278558754, 163.7439087110442],
[87.86080841858478, 99.02332045029644, 78.44170076949185, 61.12980866505168, 172.74817246876194, 166.06685025456858, 174.61773023584269, 179.18741780973576],
[73.5729192252481, 98.45457590849806, 79.89749001663387, 58.37952711680239, 171.95314420827643, 161.07346895820075, 159.19930272336563, 176.5839665060397],
[77.09425290795956, 91.97420096284596, 75.69413571689861, 54.012967132270454, 172.8799786716053, 154.84035809250548, 158.25107588745936, 161.39005214806022],
[115.6360214660742, 107.56175660029291, 56.4632782926502, 55.22135232489106, 166.06639559829554, 168.3839968582741, 168.40950622714982, 173.29930495263432],
[83.16104045929458, 94.44269698304558, 74.94941169280524, 58.23120288942816, 167.25613292131536, 161.49524964705842, 158.51659777815232, 160.04378233713584],
[112.85790225547265, 99.97769456505102, 33.486744718474576, 50.99940344065596, 161.2539363218046, 165.2213847208534, 173.82144497727447, 159.7582486696675],
[77.42957323785198, 84.9094562459833, 72.13116542410874, 78.65117313932845, 172.18087459357386, 170.2904433053635, 161.54120054644707, 175.05765438672242],
[118.68687650903308, 112.40080924954444, 4.7608729187922405, 19.57913015815162, 167.63940350409678, 178.0753215610297, 167.78415064715958, 158.77651459499165],
[30.93197755990579, 32.56132528543088, 144.54173135243153, 144.98385823868048, 171.0773661002086, 164.30047164254768, 161.07212709518632, 165.50207470231442],
[34.147122808340406, 35.176206164160156, 152.66939317300594, 148.81054313394333, 161.77901437101022, 161.80387719197506, 169.11476543677642, 167.8371851435744],
[37.62705525227641, 38.76579483125054, 151.51079320760718, 150.07942556310564, 165.98129192424486, 167.04183790932944, 169.59944057894694, 173.64229697736903],
[43.81824826309406, 44.631469713405544, 145.44115145679194, 139.27434352055943, 169.89755037585272, 167.62382367971506, 175.2727437180801, 171.86373007398774],
[44.05729595623198, 43.78712120592072, 149.93700856485023, 148.80141770833822, 157.58830863182916, 162.90637899035573, 176.18625845899217, 171.43430278055996],
[47.60056480533439, 50.99606200703988, 145.01328388607934, 138.089122665545, 161.02566217681394, 164.5065694818513, 179.46200684946302, 176.45401789516723],
[52.240135071275816, 53.85195887756588, 137.0293506385811, 136.6317491763493, 156.0846903427295, 159.95560617432182, 173.42220085272902, 171.6419068713373],
[51.9486974410016, 51.11522834549808, 140.9401454950272, 138.64409741824193, 156.58457761200052, 163.9593868437287, 177.71543279406978, 171.97705635962308],
[45.95916991813041, 49.42222827682974, 149.2108482839716, 138.41720801787244, 169.55844748610343, 162.10041603294246, 177.5222987481556, 169.80860398259685],
[56.223050385362896, 54.62796875490928, 141.26031924922285, 139.7218743249421, 170.6055725641862, 174.32075449386323, 177.85807252668076, 172.86994424701223],
[61.43594186193612, 61.046604035093274, 132.55298237370906, 128.79832853052042, 166.26459316940347, 166.8045467712507, 172.4755615200346, 165.12198508962177],
[57.543561360680336, 57.163192773874954, 134.52852077958406, 134.46018783099117, 171.12558886998147, 168.69624752416408, 166.7968120069173, 167.6113081593799],
[63.4250257299784, 70.23390301898478, 124.51279439106995, 124.98132746287814, 162.29939328209295, 164.67856656610243, 166.7841892482545, 173.20063174578226],
[67.51346999476499, 72.66163337501985, 128.72763144905457, 126.12793537616531, 169.35264290843824, 168.84991632846786, 166.69878913013306, 167.00660882858628],
[72.3165133984293, 76.45772996685416, 122.95484192210586, 122.61774204889014, 166.9586810534108, 168.04547892213213, 166.32879272759484, 170.95003253225195],
[74.86827754667519, 78.40118133303547, 119.82610511474024, 119.27133228612144, 168.2799656544288, 168.22421494120323, 165.75524248374373, 166.04686814412528],
[75.05125813544996, 77.61169400917596, 116.74438534922277, 119.04523106177763, 169.08211862789804, 169.317418720333, 167.7289442962581, 167.5407282130446],
[75.18891454565546, 77.74398315834407, 118.01511906445691, 119.30424249108043, 169.86311159121396, 169.81282260046643, 168.19056750876047, 167.62564807690157],
[74.46732971872872, 79.89086984269909, 118.74417612914112, 117.34931871588239, 170.3306626945283, 169.95618387545895, 164.33230056736042, 169.92466189486876],
[73.10091453665596, 79.94972351030853, 118.20601032292126, 117.65949486735255, 165.0931349334918, 167.40440420948644, 167.9858197558656, 176.22217814286827],
[70.25623980697705, 74.01858689369992, 123.62628635471293, 124.2850261834559, 170.12988452729903, 168.81224149162483, 171.5894715407197, 170.7344197966965],
[63.87355740795595, 69.92849402198905, 129.2661640111359, 126.47260817023371, 168.5785069466557, 168.86479880568433, 163.52668831254428, 166.7176207828491],
[64.99844805874204, 69.26610789419613, 130.01644572255339, 128.1842120985835, 168.08074429985083, 169.69290651915392, 166.12007533714345, 169.03923026378445],
[58.10928124354386, 59.936478967253734, 129.56567851340225, 133.17604801001835, 160.6721388615232, 166.65681948226145, 166.11739819047, 167.33370492110316],
[58.082549709677174, 58.674822069758505, 134.57947360736551, 136.844445121272, 166.61147235285125, 172.69067612223867, 164.31531480333297, 166.8872137902683],
[56.15814097595381, 52.57125039518583, 139.19351570294043, 141.4391808418791, 164.2690784998786, 167.0563902459751, 169.320117773567, 166.371683495053],
[54.11148425534333, 53.02614391987698, 141.76731218521493, 142.13678169723332, 170.20126292694707, 171.69397479968717, 165.93441834278505, 167.09528628762217],
[52.00601094511856, 50.45371206287362, 143.54043801356858, 143.38824036301622, 167.89757962794351, 169.11155581506992, 167.2849340340016, 167.49223650582366],
[42.705989069479024, 40.69904099877101, 150.6277446135127, 150.52519720999814, 164.44070781503402, 161.1246721553456, 164.86022726005845, 164.8444320538946],
[43.77099543329545, 40.047018088522854, 151.7586441647226, 151.95465200301697, 169.68247821130217, 168.34163317871793, 163.60684569316086, 161.20680007255555],
[48.77116646963689, 44.88389124401591, 143.42938052727106, 149.32716731090414, 157.48530185089953, 169.4679953571368, 166.8505606391571, 160.30572442595408],
[50.090429242914915, 49.83060242896763, 96.88478614676931, 100.47385903822305, 170.9444934973656, 168.68663667684996, 93.66154005860756, 83.11051004108333],
[35.26222572707309, 37.492202928760335, 148.5826641753483, 152.6168133092649, 145.52922619339674, 165.05081217576495, 159.7129294811232, 165.93651557275757],
[39.589583307090734, 39.91057687642735, 154.88715309477632, 151.59316855641936, 167.4367133663234, 165.17664827340116, 163.25060691680093, 162.3092207788104],
[36.23679473996376, 36.044845542892745, 152.27233348997387, 152.7401985138133, 157.30046881572235, 156.73599728347625, 161.4957792152943, 162.18527181300848],
[33.74486312707589, 33.91308805433905, 153.88830796781508, 153.41476459634814, 157.6160117645015, 163.902023638008, 163.09374838916264, 159.83765885437612],
[33.927316141500434, 34.365390575246586, 147.12725211165136, 153.6410193473446, 146.60646372179144, 164.38101869357456, 160.42983635793715, 162.122716795527],
[35.685241920172714, 34.185581668547755, 151.52180813447856, 155.78765279364302, 150.8101760345166, 166.37058029841543, 158.57967648422027, 159.41889653617451],
[34.287522440104716, 33.87589465577064, 160.07510607613935, 156.6307488034735, 169.83797771912512, 167.44424795046072, 163.03686450158267, 159.715763938277],
[32.12621201218218, 33.39430483323477, 160.96158091435106, 156.75213052138108, 167.8447453951992, 168.33012977986775, 161.32570718549192, 160.51334798240669],
[33.12322517121471, 34.48414901536369, 158.91521164961787, 155.8336634458719, 167.51628244158618, 164.4361423297073, 161.60328349421403, 159.19532586714345],
[32.25161427678373, 32.63525321587879, 160.2847410333602, 156.39446355165694, 168.77629170762694, 164.7312145262213, 161.91572184816914, 159.27137911060646],
[33.60914776957325, 33.632450148717496, 157.8068557967021, 153.82254481846874, 167.34829790183, 164.16007303895523, 159.95569252771497, 158.8470134785508],
[53.964667314072464, 50.28635253288498, 99.26855759216357, 100.02480129146232, 166.72206562143208, 163.7764251920939, 95.7668640387572, 101.75864940499969],
[35.27094112172773, 33.76785503728567, 154.8028692708318, 155.60346752780606, 166.99043419998156, 167.1622995511395, 157.9699618093254, 158.0110820005504],
[34.41523542393506, 33.54324689399283, 156.82272484847422, 155.75748880540664, 167.3106954816221, 167.9822963350602, 163.83264844032365, 160.50553391870181],
[35.45616636593782, 34.29250539246748, 154.848968562649, 153.66585432132928, 169.47854101971808, 166.69239030656914, 163.68634418479692, 162.01834876183145],
[36.90482701979156, 34.64548487665438, 152.16861298789073, 155.13917850780825, 171.34727146294892, 174.13778429295758, 159.00807923226142, 157.41710933224127],
[33.39762012643948, 33.656233029589856, 154.29673563314444, 152.74894860748094, 164.6975569222596, 164.97993764858037, 163.2792303699833, 160.81700248256539],
[31.400736456093465, 32.90412890561363, 155.60410762118875, 155.3588841491543, 155.89906283558872, 167.97551950188873, 159.89358565918465, 159.20854050432214],
[34.23946888442702, 34.9397109965947, 153.94457361068788, 152.53473223588418, 165.26701424059954, 166.38341170014678, 158.52800689117242, 161.2363136513241],
[34.10369800794362, 33.07892525180125, 157.25000065198242, 154.7761985955882, 167.67435039643735, 167.02994072959976, 159.0083171644824, 157.4916558954035],
[34.70650696005048, 33.41532490587363, 155.9752759139312, 155.0315131074516, 165.73385114501602, 167.01074015678878, 163.73006350905067, 161.00348623706546],
[33.33193930496211, 33.4902618059734, 158.449758885399, 153.94454348741252, 170.2357359758292, 167.71467179518413, 161.88556291652998, 157.37033819457773],
[70.08957256827028, 61.7514741458493, 81.4049548121656, 85.21156205190903, 173.0370732191965, 170.98976738914757, 133.86167193793395, 132.02678017360702],
[32.19967418588854, 33.16031591868705, 156.7957425008046, 152.37037372356957, 167.96181253062645, 165.08990309883376, 163.08104530426596, 160.84135318560962],
[35.141290261542125, 33.60223987347286, 152.11983052046335, 152.32505688594642, 166.21416459927153, 166.73334107713796, 160.08011494186744, 156.52124188010782],
[32.723649215604134, 33.300766583131804, 154.7863912057937, 151.68350809553357, 166.681528845939, 165.81230720807568, 160.24836463172875, 157.0236938188465],
[32.20234313170978, 33.10532792474659, 159.4385924423602, 153.93038661297078, 170.45563158395905, 167.14699208411884, 163.7751919377637, 159.6021663141685],
[33.318822642947694, 34.259211794521384, 148.77167939642206, 147.9521127831266, 148.66153028023814, 158.067346155543, 157.3942375951639, 154.08278925653804],
[29.849167488049616, 32.46279167554499, 156.1272343418733, 150.19770861372243, 149.30614484172895, 147.07867889938584, 162.503313831841, 163.77570465310998],
[31.76362010742378, 32.25853409990228, 152.31024392976454, 152.05214599932984, 155.75233978115216, 165.93134865436662, 159.30336885530414, 159.7356174019213],
[30.37853111218629, 30.844722419099032, 153.0734610019918, 152.19904606923558, 150.0947372681662, 158.2059203996478, 165.80651019503162, 165.72929772830534],
[29.44100499798838, 30.800154915589026, 157.17521900088465, 153.0701480629686, 155.4299399879056, 157.25518928284154, 162.33348057998643, 161.64404988826976],
[35.18278237253519, 32.56570027607743, 147.7232853471017, 152.73770584299595, 153.5915914206767, 168.29407548047467, 156.49154663469756, 154.39806038758073],
[34.75761889274472, 32.94606578764251, 147.34480831892807, 152.95222216739936, 159.40723968534553, 172.18573336997153, 160.25694268782715, 157.31247022791047],
[34.34097347355785, 31.518651035798477, 154.1209336118222, 150.6437244631343, 168.66805135089484, 164.40612822670633, 155.4314333698704, 150.4789046474251],
[81.22434111120897, 71.32966756509926, 59.94205162491267, 72.29060968634423, 163.94457237928899, 166.34782263422235, 158.21448025318796, 155.08499434659285],
[35.01382048199337, 32.28313338471699, 140.22913115430788, 146.23356097998786, 144.8658167737374, 164.1654064557235, 137.3332025918887, 131.9817379731429],
[47.64677569956896, 45.36411498822721, 113.23161575633532, 116.91576552840525, 169.18515169454656, 169.50186602381154, 104.47467247274056, 101.4066955598575],
[33.77968525411542, 32.160389260705855, 129.6339259105839, 138.3679640185615, 148.57785500465934, 169.98144286256382, 117.90700334111065, 112.5584168826293],
[68.85279876561418, 59.79351267992632, 92.77873861287378, 98.71495728428249, 168.7356125625593, 171.71860746296946, 90.5947262991748, 80.68715817754389],
[122.02287145860635, 121.62315747888633, 34.07943550706121, 15.377677283486946, 173.81750767706967, 155.79168436736245, 153.09522151117991, 138.96046545519505],
[91.44233316282764, 79.7094161309358, 49.30690128457031, 72.78333284624573, 174.56244738962275, 178.2376412027568, 68.35995402771391, 77.40192038386084],
[90.9545580130343, 84.5543549734498, 59.5122903388112, 66.23831685864668, 171.3433672486204, 169.09191600189834, 50.36306546640504, 47.36683769162553],
[82.93187881733343, 81.92207269824036, 65.25689539490483, 68.02626795731848, 172.31019154624292, 173.53861000094034, 55.85704559699508, 59.45224580216501],
[72.45173872062223, 70.51941838752293, 77.49972786617255, 76.44360607193762, 170.74962338221056, 167.2537962305963, 50.47654512222324, 49.59547292338956],
[71.45527082112517, 66.14571568485458, 81.36136563010183, 80.89133865558648, 173.49728779196693, 164.50053042015432, 80.26371078667722, 75.51506043456533],
[147.98567118141432, 163.05183490727495, 1.4773587440421008, 10.48668274765722, 173.26996281490722, 152.53848822706465, 152.69384042723016, 171.2102381762203],
[173.52396010412275, 172.322328107979, 50.98448717837289, 31.200637322470463, 33.172950047462734, 140.7810766261323, 171.91528653346845, 179.20893387433904],
[177.42916548477066, 171.13284004586313, 83.01339886694359, 25.47143245834291, 56.79727946912241, 143.88839393403546, 172.1148186252297, 179.41601176377995],
[174.0613622779273, 177.58970836355263, 100.82351688431967, 22.046920020229724, 88.58141460796429, 140.39081021893082, 178.0653497396268, 164.78738847098765],
[169.65496105209647, 179.79810458912908, 18.425928202572816, 21.81027466408611, 163.86702155031273, 141.30739842505974, 168.07213050039744, 161.61428573050605],
[170.5732277124396, 175.79198913987207, 14.956732357226342, 19.87175525331373, 172.69688124160575, 152.12217767559653, 170.5683507694983, 149.6165430665381],
[174.14048406247085, 177.0565520833804, 50.760730098636685, 21.605863474471747, 29.24200318628974, 157.10883369667647, 176.09210068240154, 170.80629801062167],
[179.66115559403895, 175.2927976557049, 32.76637475807283, 22.172694387753367, 2.850718664499209, 167.95793285829654, 176.2434994505248, 179.94405302101626],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],
[179.05455421367003, 179.89996398922779, 26.92698282969558, 20.84150072002559, 158.2553748442467, 166.40449164906244, 179.6134679240023, 178.81684717786555],

]



eg = [169.65496105209647, 179.79810458912908, 18.425928202572816, 21.81027466408611, 163.86702155031273, 141.30739842505974, 168.07213050039744, 161.61428573050605]
# Sanity test classification 
# classify_pose([8.390697637127042, 13.364568331384618, 16.49759248897499, 153.50000646379374, 173.20291493738577, 199.52935190007366, 179.00845878279233, 198.25172013734928], classifer='SVM')
classify_pose(eg, classifer='SVM')

for x in fuckme:
    print(classify_pose(x, classifer='SVM')[0])


Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Cobra
Cobra
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Tree
Tree
WarriorII
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Chair
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Cobra
Cobra
Neutral
Neutral
Neutral
Neutral
Cobra
Cobra
Cobra
Cobra
Cobra
Cobra
Cobra
Cobra
Cobra
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Tree
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Tree
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Tree
Neutral
Neutral
Neutral
Neutral
Neutral
Cobra
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Tree
Neutral
Tree
Neutral
Tree
Tree
Tree
Tree
Tree
Neutral
DownDog
DownDog
DownDog
Neutral
Neutral
WarriorII
WarriorII
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neutral
Neut

In [ ]:
pca = PCA(n_components = 2)
X_train_pca = pca.fit_transform(X_train)

def extract_pca_2d(example):
    example = pd.DataFrame(np.array(example).reshape(1, -1), columns=columns[1:])
    return pca.transform(example)

def unnormalize_cords(x, y, fw, fh):
    return tuple(np.multiply([x, y], [fw, fh]).astype(int))

# Getting the ideal angles into a dataframe
## Currently the method just gets the ideal images and finds the angles in those images and uses that as the ideal example.
## A more sophisiticated method may be getting a large amount of ideal examples and averaging over all of their angles to get a mean ideal angle for each key point. Ideal

# Edits: 
## There are now two ways of calculating the ideal angle - one is more sophisticated and splits up WarriorII and Tree into 2 different 'subclasses' one for facing left and one for facing right. This was done as the score someone would recieve for doign these exercises wouldn't be accurate as we previously didn't take into account which variation they did it

## The old way of getting the average angles still exist and we simply just need to the toggle the `OLD_IDEAL_ANGLES` variable.

In [ ]:
OLD_IDEAL_ANGLES = False 

if OLD_IDEAL_ANGLES:
    # Depricated since we're now differentiate left and right warrior 
    ''' Might end up going back to this'''
    ideal_angles = pd.read_csv('../DemoImages/Ideal_Angles.csv', header=None)
    ideal_angles.columns = columns 

    ideal_angles_map_single = {pose : ideal_angles[ideal_angles['class'] == pose].values.tolist()[0][1:] for pose in classes}

    ''' Finding the average for all poses'''
    ideal_angles_map_average = {pose : combined_train[combined_train['class'] == pose_idx].mean(axis=0).tolist()[1:] for pose_idx, pose in enumerate(classes) }

else:
    ideal_angles = pd.read_csv('../DemoImages/joint_angles.csv', header=None)
    ideal_angles.columns = columns
    ideal_angles_map_single = {pose : ideal_angles[ideal_angles['class'] == pose].values.tolist()[0][1:] for pose in feedback_classes}
    ideal_angles_map_single['Neutral'] = []
    
    ''' Finding the average for all poses except WarriorII and Tree. These need to be dealt in a special case explained in the next section'''
    ideal_angles_map_average = {pose : combined_train[combined_train['class'] == pose_idx].mean(axis=0).tolist()[1:] for pose_idx, pose in enumerate(classes) if pose not in {'WarriorII', 'Tree'}}

    ''' Splitting Warrior into Warrior L and Warrior R -- Setting the threshold to 170 gives the most equal 5050 split (180:196) and makes sense'''
    w2index = classes.index('WarriorII')
    Right_WarriorII = combined_train[(combined_train['class'] == w2index) & (combined_train['l_knee'] > 170)]
    Left_WarriorII = combined_train[(combined_train['class'] == w2index) & (combined_train['l_knee'] <= 170)]
    

    ideal_angles_map_average['WarriorII_R'] = Right_WarriorII.mean(axis=0).tolist()[1:]
    ideal_angles_map_average['WarriorII_L'] = Left_WarriorII.mean(axis=0).tolist()[1:]

    ''' Splitting Tree into Tree L and Tree R -> The split of is 100:250 (L:R) even while changing the degree threshold this value doesn't change much.'''
    ''' Potentially gonna have to split into Arms-Up Tree and Arms-Down Tree since the training data may include both poses with arms up and poses with arms down'''
    tree_idx = classes.index('Tree')

    Left_Tree_Down = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] <= 90) & (combined_train['l_shoulder'] <= 90) ]
    Right_Tree_Down = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] > 90) & (combined_train['l_shoulder'] <= 90) ]
    Left_Tree_Up = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] <= 90) & (combined_train['l_shoulder'] > 90) ]
    Right_Tree_Up = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] > 90) & (combined_train['l_shoulder'] > 90) ]


    # min_shoulder = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] > 120)]['l_shoulder'].min()
    # max_shoulder = combined_train[(combined_train['class'] == tree_idx) & (combined_train['l_knee'] > 120)]['l_shoulder'].max()

    # print(min_shoulder, max_shoulder)
    
    ideal_angles_map_average['Tree_R_D'] = Right_Tree_Down.mean(axis=0).tolist()[1:]
    ideal_angles_map_average['Tree_L_D'] = Left_Tree_Down.mean(axis=0).tolist()[1:]
    ideal_angles_map_average['Tree_R_U'] = Right_Tree_Up.mean(axis=0).tolist()[1:]
    ideal_angles_map_average['Tree_L_U'] = Left_Tree_Up.mean(axis=0).tolist()[1:]
    
ideal_angles_map = ideal_angles_map_single
ideal_angles_map['Tree_L_D']

In [ ]:
def cosine_similarity(angle1, angle2, difficulty):
    """
    Calculates the cosine similarity between two angles in degrees.
    """
    angle1_rad = np.deg2rad(angle1)
    angle2_rad = np.deg2rad(angle2)
    cos_sim = np.cos(angle1_rad - angle2_rad)

    # Rescale the cosine similarity to be between 0 and 1, with values closer to 0 indicating a worse match
    sim_rescaled = (cos_sim + 1) / 2
    
    ''' Returing the score raised to the n'th power, where n determines how sensitive the score is.'''
    return sim_rescaled**difficulty

# Define the x-axis values (the angle indexes)
x = joint_idx_map.values()

# Set up the figure and axis for the plot
fig, ax = plt.subplots()

# Plot the cosine similarity values for each pose (excluding 'Neutral')
if OLD_IDEAL_ANGLES:
    poses = classes
else:
    poses = feedback_classes

for pose in poses:
    if pose == 'Neutral':
        continue
    
    cosine_similarities = [cosine_similarity(single, avg, 1) for single, avg in zip(ideal_angles_map_single[pose], ideal_angles_map_average[pose])]
    ax.plot(x, cosine_similarities, label=pose)

# Set the plot title and labels for the x- and y-axes
ax.set_title('Cosine Similarity between Single and Average Ideal Angles for Each Pose (W2 & Tree Split)')
ax.set_xlabel('Joint')
ax.set_ylabel('Cosine Similarity')

# Set the y-axis limits to be between 0 and 1
ax.set_ylim([0.4, 1])
ax.legend()
plt.show()

In [ ]:
if OLD_IDEAL_ANGLES:
    poses = classes
else:
    poses = feedback_classes
print(list(joint_idx_map.values()))
for pose in poses:
    
    cosine_similarities = [cosine_similarity(single, avg, 1) for single, avg in zip(ideal_angles_map_single[pose], ideal_angles_map_average[pose])]
    print(pose)
    print(f'Single Example Angles: {ideal_angles_map_single[pose]}')
    print(f'Average over training set: {ideal_angles_map_average[pose]}')
    print(f'Cosine Similarities: {cosine_similarities}')
    print()

# Some extra data post-processing
## WarriorII and Tree are unique in the sense that the ideal angles are different if you're doing the left or right variation of the exercise
### My method of fixing this is, for each example we detect if it's the left variation or right variation and create a seperate entry in the map depending on that.

In [ ]:
''' Suprisngly using the same angle of 120 as a rough mid point between allowing 30degrees away from a 90degree is reasonable for both W2 and Tree
But I will leave the explicit code in to add to the readabilty.'''
def is_left_pose(pose, angles):

    ''' The standard index of left knee in the angles'''
    l_knee = 6 
  
    if pose == 'WarriorII':
        return angles[l_knee] <= 170 
    elif pose == 'Tree':
        return angles[l_knee] <= 90
    else:
        raise Exception("Only poses supported are 'WarriorII' and 'Tree'")
    
def is_down_tree(angles):
    l_shoulder = 0
    return angles[l_shoulder] <= 90

# Caluclate the total score of a users score. Currently using trivial MSE -- will get more sophisitcated as development continues 

In [ ]:
''' This didn't work haha'''
def calculate_pose_score(ideal_example, user_example):

    tot = 0 
    for x in zip(ideal_example, user_example):

        tmp_score = 100 * (1 - (np.abs((x[0] - x[1]))/360))
        tot += tmp_score
        # print(tmp_score)
    
    # print(tot//len(ideal_example))

''' Using Cosine Similarty Method --- Not exactly the best implementation

Convert the two lists of angles into numpy arrays, and then we calculate the dot product of the two arrays and divide it by the product of their norms. 
This gives us the cosine similarity. Finally, we return a score between 0 and 100 by adding 1 to the cosine similarity and multiplying it by 50. 
The result will be a number between 0 and 100, where 100 indicates that the vectors are identical and 0 indicates that the vectors are completely dissimilar.'''
def calculate_pose_score(current_ideal_angles, pose_relevant_landmark_angles):
    current_ideal_angles = np.array(current_ideal_angles)
    pose_relevant_landmark_angles = np.array(pose_relevant_landmark_angles)

    cos_sim = np.dot(current_ideal_angles, pose_relevant_landmark_angles) / (np.linalg.norm(current_ideal_angles) * np.linalg.norm(pose_relevant_landmark_angles))

    return ((cos_sim + 1) * 50)/100


def cosine_similarity(angle1, angle2, difficulty):
    """
    Calculates the cosine similarity between two angles in degrees.
    """
    angle1_rad = np.deg2rad(angle1)
    angle2_rad = np.deg2rad(angle2)
    cos_sim = np.cos(angle1_rad - angle2_rad)

    ''' Returing the score raised to the n'th power, where n determines how sensitive the score is.'''
    sim_rescaled = abs(cos_sim) ** difficulty
    
    ''' Returning weather the original cosine similarity was positive or negative to indicate which direction to turn'''
    return (cos_sim < 0, sim_rescaled)

### Helper Functions

In [ ]:
''' Doesn't work as intended'''
def display_error_image(frame_width, frame_height):
    # Create a black image to use as the background
    image = np.zeros((frame_height, frame_width, 3), np.uint8)

    # Add text to the image
    text = "Cannot detect user, please get in frame"
    font = FONT
    font_scale = 2
    font_thickness = 5
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_x = int((frame_width - text_size[0]) / 2)
    text_y = int((frame_height + text_size[1]) / 2)
    cv2.putText(image, text, (text_x, text_y), font, font_scale, WHITE_TEXT, font_thickness, LINE)

    # Display the image
    cv2.imshow("ZenAI", image)
    time.sleep(1)

''' Render pose estimated from mediapipe'''
def render_skeleton(image, landmarks):
    mp_drawing.draw_landmarks(
        image, 
        landmarks,
        mp_pose.POSE_CONNECTIONS,                
        mp_drawing.DrawingSpec(color=(50, 145, 168), circle_radius=2, thickness=2),
        mp_drawing.DrawingSpec(color=(209, 192, 42), circle_radius=2, thickness=2)
    )

''' Extracting the angles for each of the relevant joints in the frame'''
def extract_joint_angles(pose_landmarks):
    ''' Converting landmarks into angles'''
    pose_relevant_landmark_angles = []
    # Going through all relevant landmarks, extracting their key angles
    # Calculating the angle then adding to array 
    for i1, i2, i3 in angle_idxs_required:
        
        fst = (pose_landmarks.landmark[i1].x, pose_landmarks.landmark[i1].y)
        snd = (pose_landmarks.landmark[i2].x, pose_landmarks.landmark[i2].y)
        thrd = (pose_landmarks.landmark[i3].x, pose_landmarks.landmark[i3].y)
        
        pose_relevant_landmark_angles.append(calc_angle(fst, snd, thrd))

    pose_relevant_landmark_angles_visual = np.around(pose_relevant_landmark_angles, 2).astype(str).tolist()

    return pose_relevant_landmark_angles, pose_relevant_landmark_angles_visual

''' Extracting the x, y cords of each joint'''
def extract_joint_cords(pose_landmarks):
    # Getting cords of the landmarks FOR ANGLES WE CALC'D CORDS FOR. 
    # If any any of this landamrks have a visbility < MIN_DETECTION_CONFIDENCE. 
    # Display an error / throw an error saying you need the whole body in the frame
    pose_relevant_landmark_cords = [] 
    for _, idx, _ in angle_idxs_required:
        if idx in skip_landmark:
            continue
        
        current_landmark = pose_landmarks.landmark[idx]
        
        pose_relevant_landmark_cords.append([current_landmark.x, current_landmark.y])

    return pose_relevant_landmark_cords

def create_live_video_display(image, pose_landmarks, classified_pose, classified_pose_confidence, frame_width, frame_height):
    #Revert image color 
    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    render_skeleton(image, pose_landmarks)
    
    display_text = 'Neutral' if classified_pose == 'Neutral' else f'Confidence: {classified_pose} {classified_pose_confidence*100:.2f}%'
    
    cv2.putText(image, display_text, unnormalize_cords(0.1, 0.1, frame_width, frame_height), FONT, 2, (125, 0, 0), 2, LINE) 

    return image

    
def gradient(score):

    """Converts a score between 0 and 1 to an RGB value that is a gradient between red and green."""
    r = int(max(0, min(255, (1 - score*3) * 255)))
    g = int(max(0, min(255, score * 255)))
    b = 0
    return (b, g, r)



def generate_user_pose_feedback(joint, user_angle, ideal_angle, score, is_greater_than_90, joint_idx, pose):
    """
    Generates feedback for the user's pose based on the joint, score, and angle difference.
    """
    if score >= 0.95:
        return f"Great job! Your {joint} is in the ideal position."
    else:
        rough_ideal_angle = min([0, 20, 45, 60, 90, 120, 135, 160, 180], key=lambda x: abs(x-int(ideal_angle)))
        rough_angle = round(float(user_angle), -1)


        if score <= 0.3:
            severity = 'a lot'
        elif 0.3 < score <= 0.6:
            severity = 'a fair bit'
        else:
            severity = 'a little'


        action = [pose][joint_idx][0] if is_greater_than_90 else vocab_dict[pose][joint_idx][1]
        
        common_error, common_fix = vocab_dict[pose][joint_idx][2].split(',')

        feedback_suggestion = {
            'raw' : f"{joint} | U: {rough_angle} / {user_angle} | G: {rough_ideal_angle} / {ideal_angle:.2f} | S: {score:.2f} | P: {pose}",
            'formatted' : [f'Your {joint} is out of place.', 
                           f'You should aim to get your {joint} to roughly {rough_ideal_angle}',
                           f'and it\'s currently around a {rough_angle}',
                           f'You can do this by {action} your {joint} {severity} | S: {score:.2f}',
                           f'A common error here is: {common_error}',
                           f'{common_fix}'
                           ]
        }

        return feedback_suggestion


def calculate_center_of_gravity(pose_landmarks, fw, fh):
    left_hip = pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
    right_hip = pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
    center_hip = (left_hip.x + right_hip.x) / 2, (left_hip.y + right_hip.y) / 2

    left_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
    center_shoulder = (left_shoulder.x + right_shoulder.x) / 2, (left_shoulder.y + right_shoulder.y) / 2

    center_of_gravity = ((center_hip[0] + center_shoulder[0]) / 2, (center_hip[1] + center_shoulder[1]) / 2)


    arrow_start = (int(center_of_gravity[0] * fw), int(center_of_gravity[1] * fh))
    arrow_end = (arrow_start[0], int(fh * 0.8))


    return (arrow_start, arrow_end)


def create_skeleton_video_display(pose_landmarks, classified_pose, classified_pose_confidence, joint_angles_rounded, joint_cords, joint_scores, elapsed_time, reference_image, ideal_angles, feedback_pose, frame_width, frame_height):
    DISPLAY_ANGLE_SCORE = True

    ''' Creating the window that displays the feedback'''
    feedback_window = np.zeros((frame_height, frame_width, 3), np.uint8)

    ''' Create a black image to use as the background'''
    black_image = np.zeros((frame_height, frame_width, 3), np.uint8)

    ''' Render the skeleton on the black image ''' 
    render_skeleton(black_image, pose_landmarks)
    
    ''' Calculate center of gravity
        and draw an arrow from the center of gravity to the floor '''
    arrow_start, arrow_end = calculate_center_of_gravity(pose_landmarks, frame_width, frame_height)
    cv2.arrowedLine(black_image, arrow_start, arrow_end, (0, 255, 0), 5)
    
    
    ''' Only display the score information of a pose if it's not a Neutral pose'''
    if classified_pose != 'Neutral':
        ''' 
        Calculating the best, worst and average scores
    
        Calculation of the overall score may be changed to include a weighted mean
        where there will be a pre-defined weight vector of joints for each pose.
        '''
        scores_list = [score[1] for score in joint_scores]
        total_pose_score = np.average(scores_list)
        best_joint_idx = np.argmax(scores_list)
        worst_joint_idx = np.argmin(scores_list)

        improvement_suggestions = dict()
        ''' Draw red circles on the joints with size proportional to the joint score ''' 
        for i, (x, y) in enumerate(joint_cords):
            diff_over_90, score = joint_scores[i]
            joint = joint_idx_map[i]
            
            radius = int((100 * (1 - score)) * 0.5)

            black_image = cv2.circle(black_image, unnormalize_cords(x, y, frame_width, frame_height), radius, (0, 0, 255), -1)

            ''' Highlighting the worst and best joint in the users pose
                & Display the joint score for each joint on the right side of the frame'''

            if DISPLAY_ANGLE_SCORE:
                if i == best_joint_idx:
                    cv2.putText(black_image, f'Best: {joint}: {score*100:.2f}% {joint_angles_rounded[i]} ', unnormalize_cords(0.53, 0.2 + (i / (1.25 * len(joint_cords))), frame_width, frame_height), FONT, 1, (0, 255, 0), 2, LINE)
                elif i == worst_joint_idx:
                    cv2.putText(black_image, f'Worst: {joint}: {score*100:.2f}% {joint_angles_rounded[i]} ', unnormalize_cords(0.53, 0.2 + (i / (1.25 * len(joint_cords))), frame_width, frame_height), FONT, 1, (0, 0, 255), 2, LINE)
                else:
                    cv2.putText(black_image, f'{joint}: {score*100:.2f}% {joint_angles_rounded[i]}', unnormalize_cords(0.60, 0.2 + (i / (1.25 * len(joint_cords))), frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)

            improvement_suggestions[i] = generate_user_pose_feedback(joint, joint_angles_rounded[i], ideal_angles[i], score, diff_over_90, i, feedback_pose)


        ''' Draw the rectangle on the left side of the screen '''
        rect_x, rect_y = unnormalize_cords(0.05, 1, frame_width, frame_height)
        rect_width, rect_height = int(frame_width * 0.05), int(total_pose_score * frame_height)
        cv2.rectangle(black_image, (rect_x, rect_y - rect_height), (rect_x + rect_width, rect_y), gradient(total_pose_score), -1)

        ''' Displaying the average of all the users scores individual angles'''
        cv2.putText(black_image, f'Pose Score: {total_pose_score*100:.2f}%', unnormalize_cords(0.42, 0.1, frame_width, frame_height), FONT, 2, WHITE_TEXT, 2, LINE)
        
        ''' Displaying how long the user has held the pose'''
        cv2.putText(black_image, f'Time held: {elapsed_time:.2f} seconds', unnormalize_cords(0.1, 0.1, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)
    
        ''' 
        Generate Imrpovements
        '''
        # for i, suggestion in enumerate(improvement_suggestions.values()):
        #     y_placement = 0.2 + (i / (1.25 * len(improvement_suggestions)))
        #     cv2.putText(feedback_window, suggestion, unnormalize_cords(0.05, y_placement, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)
        formatted_suggestion = improvement_suggestions[worst_joint_idx]['formatted']

        ''' Generate the improvement suggestion for the worst joint'''
        cv2.putText(feedback_window, improvement_suggestions[worst_joint_idx]['raw'], unnormalize_cords(0.05, 0.1, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)

        cv2.putText(feedback_window, formatted_suggestion[0], unnormalize_cords(0.05, 0.2, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)

        cv2.putText(feedback_window, formatted_suggestion[1], unnormalize_cords(0.05, 0.3, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)
        cv2.putText(feedback_window, formatted_suggestion[2], unnormalize_cords(0.05, 0.35, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)
        
        cv2.putText(feedback_window, formatted_suggestion[3], unnormalize_cords(0.05, 0.45, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)

        cv2.putText(feedback_window, formatted_suggestion[4], unnormalize_cords(0.05, 0.55, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)
        cv2.putText(feedback_window, formatted_suggestion[5], unnormalize_cords(0.05, 0.6, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)

        ''' Pie Chart indiciating how well the joint is'''
        score_angle = int(joint_scores[worst_joint_idx][1] * 360)
        pie_center = unnormalize_cords(0.1, 0.8, frame_width, frame_height)
        cv2.ellipse(feedback_window, pie_center, (50, 50), 0, 0, score_angle, (0, 255, 0), -1)
        cv2.ellipse(feedback_window, pie_center, (50, 50), 0, score_angle, 360, (0, 0, 255), -1)
        cv2.putText(feedback_window, f'{joint_idx_map[worst_joint_idx]} Score: {joint_scores[worst_joint_idx][1]*100:.0f}%', unnormalize_cords(0.1+0.05, 0.8+0.05, frame_width, frame_height), FONT, 1, WHITE_TEXT, 2, LINE)


        ''' uncommoent '''
        # with open('test_output.txt', 'a') as f:
        #     f.write(formatted[0])
        #     f.write(formatted[1])
        #     f.write(formatted[2])
        #     f.write(formatted[3])
        ''' '''
        # diff_over_90, score = joint_scores[worst_joint_idx]
        # improvement_suggestion = generate_user_pose_feedback(joint_idx_map[worst_joint_idx], score, diff_over_90)
        
    ''' Displaying the reference image in low opacity'''
    reference_image = cv2.resize(reference_image, (frame_width, frame_height))
    blended_image = cv2.addWeighted(black_image, 0.9, reference_image, 0.1, 0)

    ''' Write the joint angles on the black image ''' 
    for i, angle in enumerate(joint_angles_rounded):
        x, y = joint_cords[i]
        cv2.putText(blended_image, angle, unnormalize_cords(x, y, frame_width, frame_height), FONT, 0.5, WHITE_TEXT, 2, LINE)

    return feedback_window, blended_image


In [ ]:
''' Plotting 2D Projection'''
def plot_live_data(points, ax):
    cords = [(extract_pca_2d(x[0]), x[1]) for x in points]

    class_labels = [point[1] for point in cords]

    unique_classes = set(class_labels)
    colors = {class_label: index for index, class_label in enumerate(unique_classes)}
    mat_colors = ['b', 'r', 'g', 'c', 'm', 'y']

    ax.clear()

    for class_label in unique_classes:
        x_v = [point[0][0][0] for idx, point in enumerate(cords) if class_labels[idx] == class_label]
        y_v = [point[0][0][1] for idx, point in enumerate(cords) if class_labels[idx] == class_label]

        c_ = [mat_colors[colors[class_label]] for _ in range(len(x_v))]
        ax.scatter(x_v, y_v, c=c_, label=class_label, s=3)

    ax.legend()

# Main video / classification loop

In [ ]:
last_pose = 'Neutral'
start_time = 0

PLAY_FEED = True

''' CV2 PutText globals'''
FONT = cv2.FONT_HERSHEY_SIMPLEX
LINE = cv2.LINE_AA
WHITE_TEXT = (255, 255, 255)
BLACK_TEXT = (0, 0, 0)

CLASSIFIER = 'SVM'

MIN_DETECTION_CONFIDENCE = 0.5
MIN_TRACKING_CONFIDENCE = 0.5

VERBOSE = False
PLOT_2D = True

EXAMPLE_POSE_IMAGE = {i.replace('.jpeg', '') : cv2.imread(f"../DemoImages/{i}", 1) for i in os.listdir("../DemoImages")}
DEMO_POSE_VIDEOS = {f.replace('.mp4', '') : '../demo_videos/' + f  for f in os.listdir('../demo_videos')}
DISPLAY_DEMO = True
SCORE_DIFFICULTY = 10

frame_width = 1280
frame_height = 720

# This is just storing each frame in a real life demonstration as a projected point in 2D using pca 
# Which then decision boundaries will be plotted of the SVM model used to determin how someone moves around in a real life example
PCA_2D_POINTS = [] 


In [ ]:
if PLAY_FEED:
    with mp_pose.Pose(min_detection_confidence=MIN_DETECTION_CONFIDENCE, min_tracking_confidence=MIN_TRACKING_CONFIDENCE, static_image_mode=False) as pose:
    
        if DISPLAY_DEMO:
            cap = cv2.VideoCapture(DEMO_POSE_VIDEOS['Downdog'])
        else:
            cap = cv2.VideoCapture(0) # Captures live video feed 

        cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
        
        while cap.isOpened():
            suc, frame = cap.read() 
            if not suc: 
                print("Frame empty..")
                continue 
            
            #Recolor image 
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            try: 
                ''' Detect pose and extract angles / cords ''' 
                result = pose.process(image=image) 

                ''' If media pipe libray can't detect any poses, just display the normal image back.'''
                if not result.pose_landmarks:# or any(landmark is None for landmark in result.pose_landmarks.landmark):
                    image.flags.writeable = True 
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    correct_example_image = cv2.resize(EXAMPLE_POSE_IMAGE[classified_pose], (frame_width, frame_height))   
                    black_image = np.zeros((frame_height, frame_width, 3), np.uint8)
                    cv2.putText(black_image, "User not detected.", unnormalize_cords(0.1, 0.5, frame_width, frame_height), FONT, 3, WHITE_TEXT, 4, LINE)
                    cv2.putText(black_image, "Please get in frame.", unnormalize_cords(0.1, 0.6, frame_width, frame_height), FONT, 3, WHITE_TEXT, 4, LINE)
                    
                else:                    
                    pose_landmarks = result.pose_landmarks

                    joint_angles, joint_angles_rounded = extract_joint_angles(pose_landmarks)
                    joint_cords = extract_joint_cords(pose_landmarks)
                    
                    ''' Classification of Pose''' 
                    classified_pose, classified_pose_confidence, prediction_probabilites = classify_pose(joint_angles, CLASSIFIER)
                    print(joint_angles)
                    ''' We take the pose the user is doing to be neutral if either it's classified as a neutral pose or the confidence of the classified pose is < 70%'''
                    classified_pose = classified_pose if classified_pose_confidence >= 0.70 else 'Neutral'

                    # Get the current time and if the pose has changed, update the start time to be the current time
                    current_time = time.time()
                    if classified_pose != last_pose:
                        start_time = current_time
                        last_pose = classified_pose

                    # If the pose did just change, then this value will be 0 
                    elapsed_time = current_time - start_time

                    feedback_pose = classified_pose
                    
                    ''' 
                        It was a waste of computation to calculate the angle score for a neutral pose

                        and if the pose is Warrior or Tree we need to add another layer of detail when scoring angles 
                        as the ideal angles of someone doing a Left Tree or Right Tree are different. 
                        This makes the feedback dynamic and even allows for variation in the tree pose specifically
                    '''

                    if classified_pose == 'Neutral':
                        angles_score = []
                    elif classified_pose in {'WarriorII', 'Tree'}:
                        left = is_left_pose(classified_pose, joint_angles)
                        right = not left

                        if classified_pose == 'WarriorII':
                            if left:
                                feedback_pose = 'WarriorII_L'
                            else:
                                feedback_pose = 'WarriorII_R'
                            

                        elif classified_pose == 'Tree':
                            down = is_down_tree(joint_angles) 
                            up = not down 

                            if left and down:
                                feedback_pose = 'Tree_L_D'
                            elif left and up:
                                feedback_pose = 'Tree_L_U'
                            elif right and down:
                                feedback_pose =  'Tree_R_D'
                            elif right and up:
                                feedback_pose = 'Tree_R_U'              

                        cv2.putText(image, f'FbP: {feedback_pose}', unnormalize_cords(0.1, 0.9, frame_width, frame_height), FONT, 2, WHITE_TEXT, 4, LINE)

                    ideal_angles = ideal_angles_map[feedback_pose]
                    ''' Calculate score for current frame given the expected exercise'''
                    angles_score = [cosine_similarity(cur_angle, ideal_angle, SCORE_DIFFICULTY) for cur_angle, ideal_angle in zip(joint_angles, ideal_angles)]

                    ''' First Window '''
                    image = create_live_video_display(image, pose_landmarks, classified_pose, classified_pose_confidence, frame_width, frame_height)

                    ''' Second Window'''
                    feedback_window, black_image = create_skeleton_video_display(pose_landmarks, classified_pose, classified_pose_confidence, joint_angles_rounded, joint_cords, angles_score, elapsed_time, EXAMPLE_POSE_IMAGE[classified_pose], ideal_angles, feedback_pose, frame_width, frame_height)
                    
                    ''' Third Window -- Not used anymore'''
                    # correct_example_image = cv2.resize(EXAMPLE_POSE_IMAGE[classified_pose], (frame_width, frame_height))            
                                
                    
                    if PLOT_2D:
                        ''' Extracting 2D PCA Points -- and plotting a new point on each frame to show how the users pose change over time,'''
                        top_pc, second_pc = extract_pca_2d(joint_angles)[0]
                        PCA_2D_POINTS.append((top_pc, second_pc))
                
                # Display the three windows side by side
                # combined_videos = np.concatenate((image, black_image, correct_example_image), axis=1)
                combined_videos = np.concatenate((image, black_image), axis=1)
                combined_videos = np.concatenate((image, black_image, feedback_window), axis=1)
                # cv2.imshow("ZenAI", image)
                # cv2.imshow("ZenAI", black_image)
                # cv2.imshow("ZenAI", correct_example_image)
                cv2.imshow("ZenAI", combined_videos)

            except Exception:
                traceback.print_exc()
                continue

            # Closing the video capture  w
            if cv2.waitKey(1) & 0xFF == ord('w'):
                break
        
    cap.release() 
    cv2.destroyAllWindows()

## Attempted multi-threading to increase FPS as analyisng each frame takes some ms which can result in the video looking like it's in slow motion

In [ ]:
# import threading
# import queue


# # Define functions for each thread
# def capture_thread(cap):
#     while cap.isOpened():
#         suc, frame = cap.read()
#         if not suc:
#             print("Frame empty..")
#             continue
#         image_queue.put(frame)

# def process_thread():
#     pose = mp_pose.Pose(min_detection_confidence=MIN_DETECTION_CONFIDENCE, min_tracking_confidence=MIN_TRACKING_CONFIDENCE, static_image_mode=False)
#     while True:
#         frame = image_queue.get()
#         #Recolor image 
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
#         try:
#             ''' Detect pose and extract angles / cords '''
#             pose_landmarks = pose.process(image=image).pose_landmarks
#             joint_angles, joint_angles_rounded = extract_joint_angles(image, pose)
#             joint_cords = extract_joint_cords(pose_landmarks)
#             ''' Classification of Pose'''
#             classified_pose, classified_pose_confidence, prediction_probabilites = classify_pose(joint_angles, CLASSIFIER)
#             ideal_angles = ideal_angles_map[classified_pose]
#             ''' Calculate score for current frame given the expected exercise'''
#             angles_score = [cosine_similarity(cur_angle, ideal_angle, 5) for cur_angle, ideal_angle in zip(joint_angles, ideal_angles)]
#             ''' Extracting 2D PCA Points -- and plotting a new point on each frame to show how the users pose change over time,'''
#             PCA_2D_POINTS.append((joint_angles, classified_pose))
#             ''' We take the pose the user is doing to be neutral if either it's classified as a neutral pose or the confidence of the classified pose is < 70%'''
#             classified_pose = classified_pose if classified_pose_confidence >= 0.80 else 'Neutral'
#             ''' First Window '''
#             image = create_live_video_display(image, pose_landmarks, classified_pose, classified_pose_confidence, frame_width, frame_height)
#             ''' Second Window'''
#             black_image = create_skeleton_video_display(pose_landmarks, classified_pose, classified_pose_confidence, joint_angles_rounded, joint_cords, angles_score, frame_width, frame_height)
#             ''' Third Window '''
#             correct_example_image = cv2.resize(EXAMPLE_POSE_IMAGE[classified_pose], (frame_width, frame_height))
#             display_queue.put((image, black_image, correct_example_image))
#         except Exception:
#             # display_error_image(frame_width, frame_height)
#             traceback.print_exc()
#             continue

# def display_thread():
#     while True:
#         images = display_queue.get()
#         # Display the three windows side by side
#         combined_videos = np.concatenate(images, axis=1)
#         cv2.imshow("ZenAI", combined_videos)
#         if cv2.waitKey(1) & 0xFF == ord('w'):
#             break

# # Set up queues for passing data between threads
# image_queue = queue.Queue()
# display_queue = queue.Queue()

# # Start threads for each task
# cap = cv2.VideoCapture(DEMO_POSE_VIDEOS['Cobra']) # Captures live video feed
# capture_thread = threading.Thread(target=capture_thread, args=(cap,))
# process_thread = threading.Thread(target=process_thread)
# display_thread = threading.Thread(target=display_thread)
# capture_thread.start()
# process_thread.start()
# display_thread.start()

# # Wait for all threads to finish
# capture_thread.join()
# process_thread.join()
# display_thread.join()

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Top PC", size=14)
ax.set_ylabel("Second PC", size=14)
ax.set_title('Projecting real life example of application onto 2D', size=16)
plot_live_data(PCA_2D_POINTS, ax)

In [ ]:
PCA_2D_POINTS #Keep this here for testing. Don't change this variable

# Plotting the scatter plot of the last ran test on a real person using the application

In [ ]:
import matplotlib.pyplot as plt 

def plot_2d_projection(points, real):
    cords = [(extract_pca_2d(x[0]), x[1]) for x in points]

    class_labels = [point[1] for point in cords]

    unique_classes = set(class_labels)
    colors = {class_label: index for index, class_label in enumerate(unique_classes)}
    mat_colors = ['b', 'r', 'g', 'c', 'm', 'y']


    fig, ax = plt.subplots()

    for class_label in unique_classes:
        x_v = [point[0][0][0] for idx, point in enumerate(cords) if class_labels[idx] == class_label]
        y_v = [point[0][0][1] for idx, point in enumerate(cords) if class_labels[idx] == class_label]

        c_ = [mat_colors[colors[class_label]] for _ in range(len(x_v))]
        ax.scatter(x_v, y_v, c=c_, label=class_label, s=3)

    ax.legend()
    plt.xlabel("Top PC", size=14)
    plt.ylabel("Second PC", size=14)   

    ''' and plotting how the different classified pose of a human looks like '''
    if real:
        plt.title('Projecting real life example of application onto 2D', size=16)
    else:
        plt.title('Projecting training data onto 2d', size=16)
    plt.show()

training_data = [(X_train.iloc[i].to_list(), classes[y_train.iloc[i]]) for i in range(len(X_train))]

plot_2d_projection(PCA_2D_POINTS, real=True)
plot_2d_projection(training_data, real=False)

In [ ]:
import matplotlib.pyplot as plt 

def plot_2d_projection(points, color, real):
    cords = [(extract_pca_2d(x[0]), x[1]) for x in points]

    class_labels = [point[1] for point in cords]

    unique_classes = set(class_labels)
    colors = {class_label: index for index, class_label in enumerate(unique_classes)}
    mat_colors = ['b', 'r', 'g', 'c', 'm', 'y']

    for class_label in unique_classes:
        x_v = [point[0][0][0] for idx, point in enumerate(cords) if class_labels[idx] == class_label]
        y_v = [point[0][0][1] for idx, point in enumerate(cords) if class_labels[idx] == class_label]

        c_ = [color for _ in range(len(x_v))]
        ax.scatter(x_v, y_v, c=c_, label=class_label, s=3)

fig, ax = plt.subplots()

plot_2d_projection(PCA_2D_POINTS, 'b', real=True)
combined_testing_data = [(X_test.iloc[i].to_list(), classes[y_test.iloc[i]]) for i in range(len(X_test))]
plot_2d_projection(combined_testing_data, 'r', real=False)

ax.legend()
plt.xlabel("Top PC", size=14)
plt.ylabel("Second PC", size=14)
plt.title('Projecting real life and test data onto 2D', size=16)
# plt.show()
